<a href="https://colab.research.google.com/github/s1111e/ComputerVision/blob/main/Object_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SECTION 4 — Tracking Models (4 Methods)


In [ ]:
# 1. Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

# 2. Define project folder
import os

PROJECT_ROOT = "/content/drive/My Drive/Colab Notebooks/CV Project"
DATA_ROOT    = os.path.join(PROJECT_ROOT, "Data/kitti_tracking")

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_ROOT   :", DATA_ROOT)

IMAGE_DIR = os.path.join(DATA_ROOT, "data_tracking_image_2/training/image_02")
LABEL_DIR = os.path.join(DATA_ROOT, "data_tracking_label_2/training/label_02")
CALIB_DIR = os.path.join(DATA_ROOT, "data_tracking_calib/training/calib")

print("IMAGE_DIR:", IMAGE_DIR)
print("LABEL_DIR:", LABEL_DIR)
print("CALIB_DIR:", CALIB_DIR)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_ROOT: /content/drive/My Drive/Colab Notebooks/CV Project
DATA_ROOT   : /content/drive/My Drive/Colab Notebooks/CV Project/Data/kitti_tracking
IMAGE_DIR: /content/drive/My Drive/Colab Notebooks/CV Project/Data/kitti_tracking/data_tracking_image_2/training/image_02
LABEL_DIR: /content/drive/My Drive/Colab Notebooks/CV Project/Data/kitti_tracking/data_tracking_label_2/training/label_02
CALIB_DIR: /content/drive/My Drive/Colab Notebooks/CV Project/Data/kitti_tracking/data_tracking_calib/training/calib
Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007 (from 1)
Receiving objects: 100% (2007/2007), 79.58 MiB | 32.12 MiB/s, done.
Resolving deltas: 100% (1158/1158), done.
/content/ByteTrack
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 74.1 MB/s eta 0:00:00
  Insta

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git
!pip install scikit-image lap filterpy


In [ ]:
import numpy as np
import cv2
import os
from deep_sort.deep_sort import DeepSort

# ReID model path (DeepSORT default)
REID_MODEL = "/content/deep_sort/deep_sort/deep/checkpoint/ckpt.t7"

# Initialize DeepSORT tracker
tracker = DeepSort(
    model_path=REID_MODEL,
    max_dist=0.2,
    min_confidence=0.3,
    nms_max_overlap=0.5,
    max_iou_distance=0.7,
    max_age=30,
    n_init=3,
    nn_budget=100,
    use_cuda=torch.cuda.is_available()
)

print("DeepSORT initialized on:", "GPU" if torch.cuda.is_available() else "CPU")


In [ ]:
def xyxy_to_xywh(box):
    x1, y1, x2, y2 = box
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w/2
    cy = y1 + h/2
    return [cx, cy, w, h]


In [ ]:
def run_deepsort_tracking(yolo_model, seq_id, output_name):
    video_path = f"{SAVE_DIR}/{output_name}.mp4"
    seq_path   = f"{IMAGE_DIR}/{seq_id}"

    # setup video writer
    frames = sorted(os.listdir(seq_path))
    frame0 = cv2.imread(os.path.join(seq_path, frames[0]))
    h,w,_  = frame0.shape
    writer = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"mp4v"), 12, (w,h))

    tracker = DeepSort(max_age=20, n_init=3, nms_max_overlap=1.0)

    for fr in frames:
        img_path = os.path.join(seq_path, fr)
        frame = cv2.imread(img_path)

        # YOLO detection
        results = yolo_model(frame, verbose=False)[0]
        dets = []
        for b in results.boxes:
            x1,y1,x2,y2 = b.xyxy[0].tolist()
            conf = float(b.conf)
            cls  = int(b.cls)
            if conf > 0.25:
                dets.append(([x1,y1,x2,y2], conf, cls))

        # DeepSORT update
        tracks = tracker.update_tracks(dets, frame=frame)

        for t in tracks:
            if not t.is_confirmed():
                continue

            x1,y1,x2,y2 = map(int, t.to_ltrb())
            tid = t.track_id

            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.putText(frame,f"ID {tid}",(x1,y1-8),
                        cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)

        writer.write(frame)

    writer.release()
    print("Saved:", video_path)


In [ ]:
# Detection models
yolo_pretrained = YOLO("yolo11n.pt")
yolo_finetuned   = YOLO("/content/drive/My Drive/Colab Notebooks/CV Project/trained_models/yolo11_kitti/weights/best.pt")

# KITTI paths (adjust if needed)
SEQ = "0001"
SEQ_DIR = f"/content/drive/My Drive/Colab Notebooks/CV Project/Data/kitti/training/image_2/{SEQ}"
SAVE_DIR = "/content/drive/My Drive/Colab Notebooks/CV Project/tracking_outputs"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
run_deepsort_tracking(yolo_pretrained, "0001", "YOLO_pretrained_DS_0001")


In [ ]:
run_deepsort_tracking(yolo_finetuned, "0001", "YOLO_finetuned_DS_0001")


# SECTION 5 — Tracking Evaluation (MOTA/MOTP/ID-Switch)

# SECTION 6 — Final Visualization + Report